In [72]:
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import implicit

# Подготовка данных

### Провайдеры 

In [73]:
df_providers = pd.read_csv('providers.csv', index_col=None)
df_providers.drop('Unnamed: 0', axis=1, inplace=True)
df_providers = df_providers.merge(df_groups, on = 'grp_id', how='left')

In [74]:
df_providers.head()

,prv_id,grp_id,prv_short_name,igrp_id
0,1638,141,Мегателл,26
1,3237,141,Райффайзенбанк Австрия,26
2,10153,141,НОРД СТАР,26
3,11804,103,Велнет,9
4,13247,30,Казкоммерцбанк,4


### Терминалы 

In [85]:
df_terminals = pd.read_csv('terminals.csv')
df_terminals.drop('Unnamed: 0', axis=1, inplace=True)
df_terminals.head()

,trm_id,trm_city_id
0,310720791183,26.0
1,310721140873,2963.0
2,310729918092,4456.0
3,310618332013,3230.0
4,310729253681,3120.0


### Транзакции 

In [76]:
df = pd.read_csv('txns', index_col=None)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.rename(columns={'txn_to_prv_id': 'prv_id'}, inplace=True)
df.head()

,phone_number,prv_id,trm_id,count,amount,last_date
0,82351539750781238,54795,112104003,1,285.897,2020-08-15 21:00:01
1,1739000057,520,115585052,1,844.660,2020-08-15 21:00:01
2,1614090218385229824,520,113142183,1,279.460,2020-08-15 21:00:01
3,2141140571107251738,26,116791246,1,94.200,2020-08-15 21:00:01
4,1844674851135093932,1166178,110888844,1,386.220,2020-08-15 21:00:01


В данном случае в пользователь у нас храниться в поле 'phone_number', а товар в поле 'prv_id'. Проиндесируем поля, что бы можно было создать разряженную матрицу

In [77]:
# df_items - каталог товаров, iitem - index товара
df_items = df[['prv_id']].drop_duplicates().reset_index(drop=True)
df_items['iitem'] = df_items.index
df_items = df_items.merge(df_providers[['prv_id', 'grp_id', 'prv_short_name']], on='prv_id', how='left')

In [78]:
# каталог пользователей, iuser - index пользователя
df_users = df[['phone_number']].drop_duplicates().reset_index(drop=True)
df_users['iuser'] = df_users.index

In [79]:
# добавляем индексы каталога пользователей
df = df.merge(df_users, on='phone_number', how='left')
# добавляем индексы каталога товаров
df = df.merge(df_items[['iitem', 'prv_id', 'grp_id']], on='prv_id', how='left')
# добавляем ид города
df = df.merge(df_terminals[['trm_id', 'trm_city_id']], on='trm_id', how='left')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23344822 entries, 0 to 23344821
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   phone_number  int64  
 1   prv_id        int64  
 2   trm_id        int64  
 3   count         int64  
 4   amount        float64
 5   last_date     object 
 6   iuser         int64  
 7   iitem         int64  
 8   grp_id        int64  
 9   trm_city_id   float64
dtypes: float64(2), int64(7), object(1)
memory usage: 1.9+ GB


Есть ряд локальных провайдеров с большими объемами, которые будут сильно искажать нам картину. Попробуем от них избавиться. Для этого найдем провайдеров с объемами больше чем p1  * 100% остальных провайдеров и при этом с меньшим количество городов чем 100% - p2 * 100% 

In [80]:
def clear_one(df, p1=0.8, p2=0.2):
    # избавляемся от транзакций операторов с большими объемами, но локальных
    prv_sum = df[['prv_id', 'count', 'amount']].groupby('prv_id').sum()
    volumed_provs = prv_sum[prv_sum['count'] > prv_sum['count'].quantile(0.8)]
    prv_city = df[df.prv_id.isin(volumed_provs.index)][['prv_id', 'trm_city_id']]\
    .groupby('prv_id')['trm_city_id'].nunique()
    
    return df[~df.prv_id.isin(anomaly.index)]


# Обучаем модель

In [91]:
def split_df(data_frame, field, frac=0.5):
    X, y = [], []
    d = data_frame.groupby(field).count()['prv_id']
    #X = data_frame.sort_values('last_date').groupby('phone_number').first()['txn_to_prv_id']
    d = d[d > 2]
    d = d.sample(int(len(d) * frac))
    #return X
    X_df = data_frame[data_frame[field].isin(d.index)]
    train_df = data_frame[~data_frame[field].isin(d.index)]
    for _, g in X_df.sort_values('last_date').groupby(field):
        X.append(g.prv_id.values[0])
        y.append(g.prv_id.values[1:])
    return train_df, X, y

In [92]:
df, X, y = split_df(df, 'phone_number', 0.05)

In [153]:
class Advisor:
    
    
    model_conf = {'K1': 0.7, 'B': 0.7, 'num_threads': 0}
    
    def __init__(self, df, rating, item_name, user_name, item_catalog, user_catalog):
        import gc
        print(len(item_catalog))
        self.df = df
        self.item_catalog = item_catalog
        self.user_catalog = user_catalog
        self.rating = rating
        self.user = user_name
        self.item = item_name
        self.sparse_matrix = self.get_sparse_matrix()
        self.model = implicit.nearest_neighbours.BM25Recommender(
            K=len(self.item_catalog), **self.model_conf)
        #del self.df
        #gc.collect()
    
    @property
    def item_user(self):
        return self.sparse_matrix.T.tocsr()
    
    def get_index_item(self, item):
        return self.item_catalog[self.item_catalog.prv_id==item][self.item].values[0]
    
    def get_item_by_index(self, index):
        return self.item_catalog.loc[index, ['prv_id', 'prv_short_name']].values
    
    def get_sparse_matrix(self):
        # список количества покупок
        data = self.df[self.rating].tolist()
        # список индексов пользователей
        items = self.df[self.item].tolist()
        # список индексов провайдеров
        users = self.df[self.user].tolist()
        # создаем раряженную матрицу
        print(f'{max(users)} - {max(items)} - {len(self.user_catalog), len(self.item_catalog)}')
        return csr_matrix(
            (
                data, 
                (
                    users,
                    items
                )
            ),
            shape = (
                len(self.user_catalog),
                len(self.item_catalog)
            ), 
            dtype=np.int32
        )   
    
    def fit(self, item_user=None):
        if not item_user:
            item_user = self.item_user
        self.model.fit(self.item_user)
        #bm25 = self.model.fit(self.sparse_matrix.T.tocsr())
        return self.model
    
    def recomend(self, item, n=3):
        iitem = self.get_index_item(item)
        r_items = self.model.similar_items(iitem, n+1)
        return [self.get_item_by_index(i) for i, _ in r_items][1:]
        
    def hitrate(self, X, y, n=3):
        hit = 0
        n = 0
        for x, Y in zip(X, y):
            for one in self.recomend(x, n):
                if one[0] in Y:
                    hit += 1
            n += 1
        return hit/n

In [154]:
predictor = Advisor(df, 'count', 'iitem', 'iuser', df_items, df_users)

3025
23064131 - 3024 - (23064132, 3025)


In [155]:
predictor.fit()

In [156]:
for i in X[5:15]:
    r = predictor.recomend(i, 3)
    print(f"{(i, df_items[df_items.prv_id==i].prv_short_name.values[0])} - {[i for _, i in r]}")

(213629, 'PariMatch.kz') - ['703-303', 'TELENET', 'Avon-продукция']
(234793, 'inDriver') - ['Фастен Рус', 'SAMPO.RU', 'Такси Сатурн']
(899483, '1xbet.kz') - ['Лифт-Профи НС', 'ТТК.Интернет', 'RCS & RDS']
(63973, 'TENNISI.KZ') - []
(96278, 'QIWI Кошелек.Пополнение') - ['QIWI Кошелек.Пополнение (безнал)', 'Qiwi Кошелек.Пополнение (Копия)', 'Фонбет']
(997932, 'Innopay') - ['QIWI Кошелек.', 'Мегафон (Россия)', 'Av_hr']
(3120, 'Dalacom & Pathword') - ['QIWI Кошелек.', 'Olimpbet', 'Altel 4G']
(20917, 'Olimpbet') - ['Снятие наличных по коду в банкоматах ККБ', 'QIWI Кошелек.', 'Activ']
(96278, 'QIWI Кошелек.Пополнение') - ['QIWI Кошелек.Пополнение (безнал)', 'Qiwi Кошелек.Пополнение (Копия)', 'Фонбет']
(102466, 'QIWI Кошелек.') - ['Av_hr', 'Innopay', 'Olimpbet']


In [157]:
predictor.hitrate(X, y)

0.2650334075723831